In [1]:
import os
import faiss
import numpy as np
from PyPDF2 import PdfReader
import pdfplumber
from docx import Document
from LLM_caller import APICaller
import tiktoken 
import openai
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
api_key = os.getenv("OPENAIKEY")
api_base = os.getenv("ENDPOINT")
api_caller = APICaller(api_key=api_key, endpoint=api_base)
client = OpenAI(api_key=api_key, base_url=api_base)

In [4]:
def extract_text_from_pdf(pdf_path):
    try:
        reader = PdfReader(pdf_path)
        text = ""
        for page in reader.pages:
            text += page.extract_text() or ""
        return text
    except Exception as e:
        print(f"ошибка PDF: {e}")
        return ""

def extract_text_from_docx(docx_path):
    try:
        doc = Document(docx_path)
        text = "\n".join([paragraph.text for paragraph in doc.paragraphs])
        return text
    except Exception as e:
        print(f"ошибка DOCX: {e}")
        return ""

def extract_text_from_txt(txt_path):
    try:
        with open(txt_path, 'r', encoding='utf-8') as file:
            return file.read()
    except Exception as e:
        print(f"ошибка  TXT: {e}")
        return ""

In [5]:
def count_tokens(text, model="cl100k_base"):
    try:
        encoding = tiktoken.get_encoding("cl100k_base")  
        return len(encoding.encode(text))  
    except Exception as e:
        print(f"ошибка токенов: {e}")
        return 0

In [6]:
def get_embedding(text, model="text-embedding-ada-002"):
    try:
        token_count = count_tokens(text, model)
        max_tokens = 8192  
        if token_count > max_tokens:
            print(f"tokens: {token_count}. Max: {max_tokens}.")
            return None
        
        response = client.embeddings.create(input=text, model=model)
        return response.data[0].embedding 
    except Exception as e:
        print(f"ошибка эмбеддинга: {e}")
        return None

In [7]:
def create_faiss_index(embeddings, vector_dim):
    try:
        index = faiss.IndexFlatL2(vector_dim) 
        index.add(np.array(embeddings, dtype='float32'))
        return index
    except Exception as e:
        print(f"ошибка Faiss индекса: {e}")
        return None

In [8]:
def split_text_into_chunks(text, chunk_size=8000):

    text = text.strip()
    if not text:
        return []
    
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    return chunks

In [9]:
def process_documents(file_paths):
    embeddings = []
    documents = []  
    for file_path in file_paths:
        if file_path.endswith('.pdf'):
            text = extract_text_from_pdf(file_path)
        elif file_path.endswith('.docx'):
            text = extract_text_from_docx(file_path)
        elif file_path.endswith('.txt'):
            text = extract_text_from_txt(file_path)

        chunks = split_text_into_chunks(text, chunk_size=3000)
        for chunk in chunks:
            embedding = get_embedding(chunk)
            if embedding is not None:
                embeddings.append(embedding)
                documents.append(chunk)

    return embeddings, documents


In [10]:
def generate_response_with_context(query, relevant_texts, api_caller, max_tokens=200, temperature=0.7):
    try:
        system_message = (
            "You are a bonsai master assistant. There are some relevant texts:\n"
            + "\n".join([f"Document {i + 1}: {text}" for i, text in enumerate(relevant_texts)])
            + "\nBased on this information answer the question:"
        )
        
        response = api_caller.call_llm(
            system_prompt=system_message,
            user_prompt=query,
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        return response
    except Exception as e:
        print(f"ошибка при генерации ответа {e}")
        return None


In [11]:
load_dotenv()

True

In [12]:
file_paths = [os.getenv("FILESPATHS")]
embeddings, documents = process_documents(file_paths)

In [ ]:
vector_dim = len(embeddings[0])

faiss_index = create_faiss_index(embeddings, vector_dim)
if faiss_index is not None:
    faiss.write_index(faiss_index, "vector_database.index")
    
    faiss_index = faiss.read_index("vector_database.index")
    
    query = "как ухаживать за сакурой?"
    query_embedding = np.array([get_embedding(query)], dtype='float32')
    
    D, I = faiss_index.search(query_embedding, k=5)  
    relevant_texts = [documents[idx] for idx in I[0] if idx != -1]

if relevant_texts:
    response = generate_response_with_context(query, relevant_texts, api_caller)
    print(response)
else:
    print("по запросу ничего не найдено.")

Уход за сакурой требует внимания к нескольким важным аспектам:

1. **Почва и удобрения**: Сакуре нужна почва, обогащённая гумусом, калием и азотом. Это помогает растению расти здоровым и крепким.

2. **Полив**: В сезон сакуру поливают половиной стакана в сутки, а зимой — реже. Важно поддерживать умеренную влажность почвы.

3. **Освещение**: Растению необходимо хорошее освещение, но следует избегать сквозняков.

4. **Формирование кроны**: Формировать крону можно начиная с 2-3-летнего возраста растения. В зависимости от желаемого стиля, крона может быть прямой, с изогнутыми ветвями или с широко раскинутыми ветвями.

5. **Обрезка**: Для миниат
